In [1]:
import pysam

In [2]:
THREADS = 24

### Introduction

In [3]:
samfile = pysam.AlignmentFile('test_data/pcawg_subset.bam', 'rb', threads=THREADS)

In [4]:
for read in samfile.fetch('1', 1, 15000):
    print(read)

HWI-ST672:204:D1HPJACXX:6:2205:8048:110927	163	0	14601	255	76M25S	0	14682	76	CAAGGAAGTAGGTCTGAGCAGCTTGTCCTGGCTGTGTGCATGTCAGAGGAAAGGGCCAAAACTGGGGGTGGGGGGGGGGGGGGGGGGGGGGCGGCGGGCGC	array('B', [33, 34, 34, 37, 37, 37, 37, 37, 34, 37, 39, 39, 27, 37, 39, 40, 40, 41, 40, 41, 40, 38, 39, 40, 41, 38, 40, 40, 36, 39, 41, 41, 40, 41, 39, 38, 34, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])	[('NH', 1), ('HI', 1), ('NM', 8), ('MD', '37C10C2C2C4G0T5T0C8'), ('AS', 159), ('RG', 'RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01')]
HWI-ST672:204:D1HPJACXX:6:2205:8048:110927	83	0	14682	255	101M	0	14601	101	GTCATGGAGCCCCCTACGATTCCCAGTCGTCCTCGTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAG	array('B', [2, 2, 2, 34, 32, 35, 32, 31, 35, 35, 35, 35, 35, 33, 32, 32, 35, 35, 35, 34, 32, 33, 35, 33, 31, 33, 31, 33, 3

In [5]:
paired_pcawg_subset = pysam.AlignmentFile('test_data/paired_pcawg_subset.bam', 'wb',
                                          template=samfile, threads=THREADS)

In [6]:
for read in samfile.fetch():
    if read.is_paired:
        paired_pcawg_subset.write(read)
        
paired_pcawg_subset.close()

In [7]:
for pileupcolumn in samfile.pileup('1', 14600, 14610):
    print(f'\ncoverage at base {pileupcolumn.pos} = {pileupcolumn.n}')
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            # query position is None if is_del or is_refskip is set
            print (f'\tbase in read {pileupread.alignment.query_name} = {pileupread.alignment.query_sequence[pileupread.query_position]}')


coverage at base 14601 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = C

coverage at base 14602 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = A

coverage at base 14603 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = A

coverage at base 14604 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = G

coverage at base 14605 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = G

coverage at base 14606 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = A

coverage at base 14607 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = A

coverage at base 14608 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = G

coverage at base 14609 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = T

coverage at base 14610 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = A

coverage at base 14611 = 1
	base in read HWI-ST672:204:D1HPJACXX:6:2205:8048:110927 = G

coverage at base 146

### Creating BAM/CRAM/SAM files from scratch

In [7]:
header = {
    'HD': {'VN': '1.0'},
    'SQ': [
        {'LN': 1575, 'SN': 'chr1'},
        {'LN': 1584, 'SN': 'chr2'}
    ]
}

In [8]:
with pysam.AlignmentFile('test_data/from_scratch.bam', 'wb', header=header) as outf:
    a = pysam.AlignedSegment()
    a.query_name = 'read_28833_29006_6945'
    a.query_sequence = 'AGCTTAGCTAGCTACCTATATCTTGGTCTTGGCCG'
    a.flag = 99
    a.reference_id = 0
    a.reference_start = 32
    a.mapping_quality = 20
    a.cigar = ((0, 10), (2, 1), (0, 25))
    a.next_reference_id = 0
    a.next_reference_start = 199
    a.template_length = 167
    a.query_qualities = pysam.qualitystring_to_array('<<<<<<<<<<<<<<<<<<<<<:<9/,&,22;;<<<')
    a.tags = (('NM', 1), ('RG', 'L1'))
    outf.write(a)

### Using samtools commands within python

In [9]:
pysam.sort('-o', 'test_data/pcawg_subset.sorted.bam', 'test_data/pcawg_subset.bam')

''

In [10]:
pysam.sort.usage()

'samtools sort: failed to read header from "-"\n'

In [11]:
pysam.sort.get_messages()

''

# API

### AlignmentFile

In [12]:
samfile = pysam.AlignmentFile('test_data/pcawg_subset.bam', 'rb', threads=THREADS)

In [13]:
samfile.check_index()

True

In [14]:
samfile.count(contig='22')

13742

In [15]:
samfile.count_coverage(contig='1', start=14500, stop=15000)

(array('L', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
samfile.find_introns(samfile.fetch(contig='1', start=10000, stop=700000))

Counter({(14829, 14969): 2,
         (15038, 15795): 1,
         (17055, 17232): 1,
         (17742, 17914): 2,
         (158674, 164262): 1,
         (165942, 168099): 1,
         (168165, 169048): 1,
         (320938, 321031): 3,
         (324345, 324438): 1,
         (535884, 566598): 2,
         (535963, 566598): 2,
         (564735, 569355): 5,
         (564717, 569367): 6,
         (564717, 569353): 8,
         (565403, 569760): 2,
         (565649, 569359): 7,
         (565877, 569362): 13,
         (565886, 569324): 14,
         (566367, 567371): 1,
         (566591, 569359): 37,
         (566755, 567106): 17,
         (566762, 569013): 24,
         (567371, 569355): 11,
         (567376, 568942): 7,
         (567468, 569579): 5,
         (567653, 569359): 2,
         (567927, 567955): 4,
         (568689, 569355): 2,
         (569173, 569359): 5,
         (569173, 569362): 24,
         (569173, 569233): 14,
         (569198, 569362): 2,
         (569198, 569359): 3,
         (

In [17]:
samfile.get_index_statistics()

[IndexStats(contig='1', mapped=77259, unmapped=0, total=77259),
 IndexStats(contig='2', mapped=48866, unmapped=0, total=48866),
 IndexStats(contig='3', mapped=59652, unmapped=0, total=59652),
 IndexStats(contig='4', mapped=186497, unmapped=0, total=186497),
 IndexStats(contig='5', mapped=28972, unmapped=0, total=28972),
 IndexStats(contig='6', mapped=41990, unmapped=0, total=41990),
 IndexStats(contig='7', mapped=27625, unmapped=0, total=27625),
 IndexStats(contig='8', mapped=18923, unmapped=0, total=18923),
 IndexStats(contig='9', mapped=37835, unmapped=0, total=37835),
 IndexStats(contig='10', mapped=33790, unmapped=0, total=33790),
 IndexStats(contig='11', mapped=47493, unmapped=0, total=47493),
 IndexStats(contig='12', mapped=44916, unmapped=0, total=44916),
 IndexStats(contig='13', mapped=8719, unmapped=0, total=8719),
 IndexStats(contig='14', mapped=33926, unmapped=0, total=33926),
 IndexStats(contig='15', mapped=17392, unmapped=0, total=17392),
 IndexStats(contig='16', mapped=56

In [18]:
samfile.get_reference_length('1')

249250621

In [19]:
samfile.get_reference_name(0)

'1'

In [20]:
samfile.get_tid('1')

0

In [21]:
samfile.has_index()

True

In [22]:
[print(read) for read in samfile.head(5)]

HWI-ST672:204:D1HPJACXX:6:2205:8048:110927	163	0	14601	255	76M25S	0	14682	76	CAAGGAAGTAGGTCTGAGCAGCTTGTCCTGGCTGTGTGCATGTCAGAGGAAAGGGCCAAAACTGGGGGTGGGGGGGGGGGGGGGGGGGGGGCGGCGGGCGC	array('B', [33, 34, 34, 37, 37, 37, 37, 37, 34, 37, 39, 39, 27, 37, 39, 40, 40, 41, 40, 41, 40, 38, 39, 40, 41, 38, 40, 40, 36, 39, 41, 41, 40, 41, 39, 38, 34, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])	[('NH', 1), ('HI', 1), ('NM', 8), ('MD', '37C10C2C2C4G0T5T0C8'), ('AS', 159), ('RG', 'RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01')]
HWI-ST672:204:D1HPJACXX:6:2205:8048:110927	83	0	14682	255	101M	0	14601	101	GTCATGGAGCCCCCTACGATTCCCAGTCGTCCTCGTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAG	array('B', [2, 2, 2, 34, 32, 35, 32, 31, 35, 35, 35, 35, 35, 33, 32, 32, 35, 35, 35, 34, 32, 33, 35, 33, 31, 33, 31, 33, 3

[None, None, None, None, None]

In [23]:
samfile.is_valid_tid(42)

True

In [24]:
samfile.lengths

(249250621,
 243199373,
 198022430,
 191154276,
 180915260,
 171115067,
 159138663,
 146364022,
 141213431,
 135534747,
 135006516,
 133851895,
 115169878,
 107349540,
 102531392,
 90354753,
 81195210,
 78077248,
 59128983,
 63025520,
 48129895,
 51304566,
 155270560,
 59373566,
 16569,
 4262,
 15008,
 19913,
 27386,
 27682,
 33824,
 34474,
 36148,
 36422,
 36651,
 37175,
 37498,
 38154,
 38502,
 38914,
 39786,
 39929,
 39939,
 40103,
 40531,
 40652,
 41001,
 41933,
 41934,
 42152,
 43341,
 43523,
 43691,
 45867,
 45941,
 81310,
 90085,
 92689,
 106433,
 128374,
 129120,
 137718,
 155397,
 159169,
 161147,
 161802,
 164239,
 166566,
 169874,
 172149,
 172294,
 172545,
 174588,
 179198,
 179693,
 180455,
 182896,
 186858,
 186861,
 187035,
 189789,
 191469,
 211173,
 547496,
 171823,
 35477943)

In [25]:
samfile.mapped

1059228

In [26]:
for read in samfile.head(n=4):
    target_read = read

In [27]:
print(target_read)

HWI-ST672:204:D1HPJACXX:6:1106:19494:193291	99	0	14716	255	101M	0	15025	101	GTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT	array('B', [31, 31, 30, 35, 37, 37, 37, 37, 39, 39, 33, 35, 39, 40, 38, 39, 40, 40, 40, 41, 40, 41, 36, 40, 38, 36, 39, 32, 35, 39, 38, 38, 40, 38, 33, 39, 35, 39, 38, 27, 36, 39, 37, 24, 26, 30, 34, 31, 31, 26, 26, 29, 29, 34, 35, 31, 34, 33, 33, 33, 26, 26, 18, 26, 33, 16, 24, 27, 27, 31, 32, 33, 24, 34, 34, 24, 27, 31, 33, 29, 31, 35, 16, 15, 10, 23, 32, 31, 32, 30, 23, 30, 33, 22, 7, 23, 19, 29, 34, 18, 29])	[('NH', 1), ('HI', 1), ('NM', 0), ('MD', '101'), ('AS', 201), ('XS', '-'), ('RG', 'RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01')]


In [28]:
print(samfile.mate(target_read))

HWI-ST672:204:D1HPJACXX:6:1106:19494:193291	147	0	15025	255	13M757N88M	0	14716	101	CTCGTGGCTCTCACTTGCTCCTGCTCCTTCTGCTGCTGCTTCTCCAGCTTTCGCTCCTTCATGCTGCGCAGCTTGGCCTTGCCGATGCCCCCAGCTTGGCG	array('B', [2, 2, 2, 34, 32, 30, 31, 35, 34, 35, 35, 34, 32, 34, 34, 32, 27, 31, 31, 23, 34, 27, 33, 32, 31, 33, 34, 35, 35, 33, 35, 33, 30, 23, 17, 34, 29, 34, 32, 27, 29, 17, 17, 32, 31, 34, 34, 28, 33, 33, 31, 27, 28, 37, 37, 37, 35, 37, 37, 36, 39, 35, 37, 36, 40, 41, 41, 40, 38, 41, 40, 41, 39, 37, 35, 27, 39, 38, 40, 38, 38, 41, 40, 38, 39, 39, 38, 37, 35, 22, 35, 35, 37, 35, 18, 35, 25, 35, 31, 31, 33])	[('NH', 1), ('HI', 1), ('NM', 0), ('MD', '101'), ('AS', 201), ('XS', '-'), ('RG', 'RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01')]


In [29]:
samfile.nocoordinate

6368

In [30]:
samfile.nreferences

86

In [31]:
samfile.references

('1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 'X',
 'Y',
 'MT',
 'GL000207.1',
 'GL000226.1',
 'GL000229.1',
 'GL000231.1',
 'GL000210.1',
 'GL000239.1',
 'GL000235.1',
 'GL000201.1',
 'GL000247.1',
 'GL000245.1',
 'GL000197.1',
 'GL000203.1',
 'GL000246.1',
 'GL000249.1',
 'GL000196.1',
 'GL000248.1',
 'GL000244.1',
 'GL000238.1',
 'GL000202.1',
 'GL000234.1',
 'GL000232.1',
 'GL000206.1',
 'GL000240.1',
 'GL000236.1',
 'GL000241.1',
 'GL000243.1',
 'GL000242.1',
 'GL000230.1',
 'GL000237.1',
 'GL000233.1',
 'GL000204.1',
 'GL000198.1',
 'GL000208.1',
 'GL000191.1',
 'GL000227.1',
 'GL000228.1',
 'GL000214.1',
 'GL000221.1',
 'GL000209.1',
 'GL000218.1',
 'GL000220.1',
 'GL000213.1',
 'GL000211.1',
 'GL000199.1',
 'GL000217.1',
 'GL000216.1',
 'GL000215.1',
 'GL000205.1',
 'GL000219.1',
 'GL000224.1',
 'GL000223.1',
 'GL000195.1',
 'GL000212.1',
 'GL000222.1',
 'GL000200.1',
 'GL000193.

In [32]:
samfile.unmapped

6368

In [33]:
samfile.pileup('1', 14500, 15000)    

### AlignedSegment

In [40]:
print(target_read.bin)
print(samfile.mate(target_read).bin)

4681
4681


In [41]:
print(target_read.cigarstring)
print(samfile.mate(target_read).cigarstring)

101M
13M757N88M


In [43]:
print(target_read.cigartuples)
print(samfile.mate(target_read).cigartuples)

[(0, 101)]
[(0, 13), (3, 757), (0, 88)]


In [44]:
target_read.compare(samfile.mate(target_read))

-53

In [45]:
print(target_read.flag)
print(samfile.mate(target_read).flag)

99
147


In [50]:
target_mate = samfile.mate(target_read)

In [51]:
target_read.get_aligned_pairs()

[(0, 14716),
 (1, 14717),
 (2, 14718),
 (3, 14719),
 (4, 14720),
 (5, 14721),
 (6, 14722),
 (7, 14723),
 (8, 14724),
 (9, 14725),
 (10, 14726),
 (11, 14727),
 (12, 14728),
 (13, 14729),
 (14, 14730),
 (15, 14731),
 (16, 14732),
 (17, 14733),
 (18, 14734),
 (19, 14735),
 (20, 14736),
 (21, 14737),
 (22, 14738),
 (23, 14739),
 (24, 14740),
 (25, 14741),
 (26, 14742),
 (27, 14743),
 (28, 14744),
 (29, 14745),
 (30, 14746),
 (31, 14747),
 (32, 14748),
 (33, 14749),
 (34, 14750),
 (35, 14751),
 (36, 14752),
 (37, 14753),
 (38, 14754),
 (39, 14755),
 (40, 14756),
 (41, 14757),
 (42, 14758),
 (43, 14759),
 (44, 14760),
 (45, 14761),
 (46, 14762),
 (47, 14763),
 (48, 14764),
 (49, 14765),
 (50, 14766),
 (51, 14767),
 (52, 14768),
 (53, 14769),
 (54, 14770),
 (55, 14771),
 (56, 14772),
 (57, 14773),
 (58, 14774),
 (59, 14775),
 (60, 14776),
 (61, 14777),
 (62, 14778),
 (63, 14779),
 (64, 14780),
 (65, 14781),
 (66, 14782),
 (67, 14783),
 (68, 14784),
 (69, 14785),
 (70, 14786),
 (71, 14787),
 (

In [52]:
target_mate.get_aligned_pairs()

[(0, 15025),
 (1, 15026),
 (2, 15027),
 (3, 15028),
 (4, 15029),
 (5, 15030),
 (6, 15031),
 (7, 15032),
 (8, 15033),
 (9, 15034),
 (10, 15035),
 (11, 15036),
 (12, 15037),
 (None, 15038),
 (None, 15039),
 (None, 15040),
 (None, 15041),
 (None, 15042),
 (None, 15043),
 (None, 15044),
 (None, 15045),
 (None, 15046),
 (None, 15047),
 (None, 15048),
 (None, 15049),
 (None, 15050),
 (None, 15051),
 (None, 15052),
 (None, 15053),
 (None, 15054),
 (None, 15055),
 (None, 15056),
 (None, 15057),
 (None, 15058),
 (None, 15059),
 (None, 15060),
 (None, 15061),
 (None, 15062),
 (None, 15063),
 (None, 15064),
 (None, 15065),
 (None, 15066),
 (None, 15067),
 (None, 15068),
 (None, 15069),
 (None, 15070),
 (None, 15071),
 (None, 15072),
 (None, 15073),
 (None, 15074),
 (None, 15075),
 (None, 15076),
 (None, 15077),
 (None, 15078),
 (None, 15079),
 (None, 15080),
 (None, 15081),
 (None, 15082),
 (None, 15083),
 (None, 15084),
 (None, 15085),
 (None, 15086),
 (None, 15087),
 (None, 15088),
 (None, 1508

In [57]:
print(target_read.get_blocks())
print(target_mate.get_blocks())

[(14716, 14817)]
[(15025, 15038), (15795, 15883)]


In [59]:
print(target_read.get_cigar_stats())
print(target_mate.get_cigar_stats())

(array('I', [101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array('I', [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
(array('I', [101, 0, 0, 757, 0, 0, 0, 0, 0, 0, 0]), array('I', [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))


In [64]:
print(target_read.get_forward_qualities())
print(target_read.get_forward_sequence())
print(target_mate.get_forward_qualities())
print(target_mate.get_forward_sequence())

array('B', [31, 31, 30, 35, 37, 37, 37, 37, 39, 39, 33, 35, 39, 40, 38, 39, 40, 40, 40, 41, 40, 41, 36, 40, 38, 36, 39, 32, 35, 39, 38, 38, 40, 38, 33, 39, 35, 39, 38, 27, 36, 39, 37, 24, 26, 30, 34, 31, 31, 26, 26, 29, 29, 34, 35, 31, 34, 33, 33, 33, 26, 26, 18, 26, 33, 16, 24, 27, 27, 31, 32, 33, 24, 34, 34, 24, 27, 31, 33, 29, 31, 35, 16, 15, 10, 23, 32, 31, 32, 30, 23, 30, 33, 22, 7, 23, 19, 29, 34, 18, 29])
GTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT
array('B', [33, 31, 31, 35, 25, 35, 18, 35, 37, 35, 35, 22, 35, 37, 38, 39, 39, 38, 40, 41, 38, 38, 40, 38, 39, 27, 35, 37, 39, 41, 40, 41, 38, 40, 41, 41, 40, 36, 37, 35, 39, 36, 37, 37, 35, 37, 37, 37, 28, 27, 31, 33, 33, 28, 34, 34, 31, 32, 17, 17, 29, 27, 32, 34, 29, 34, 17, 23, 30, 33, 35, 33, 35, 35, 34, 33, 31, 32, 33, 27, 34, 23, 31, 31, 27, 32, 34, 34, 32, 34, 35, 35, 34, 35, 31, 30, 32, 34, 2, 2, 2])
CGCCAAGCTGGGGGCATCGGCAAGGCCAAGCTGCGCAGCATGAAGGAGCGAAAGCTGGAGAAGCAGCA

In [70]:
print(target_read.get_overlap(14700, 14800))

84


In [71]:
print(target_read.get_reference_positions())

[14716, 14717, 14718, 14719, 14720, 14721, 14722, 14723, 14724, 14725, 14726, 14727, 14728, 14729, 14730, 14731, 14732, 14733, 14734, 14735, 14736, 14737, 14738, 14739, 14740, 14741, 14742, 14743, 14744, 14745, 14746, 14747, 14748, 14749, 14750, 14751, 14752, 14753, 14754, 14755, 14756, 14757, 14758, 14759, 14760, 14761, 14762, 14763, 14764, 14765, 14766, 14767, 14768, 14769, 14770, 14771, 14772, 14773, 14774, 14775, 14776, 14777, 14778, 14779, 14780, 14781, 14782, 14783, 14784, 14785, 14786, 14787, 14788, 14789, 14790, 14791, 14792, 14793, 14794, 14795, 14796, 14797, 14798, 14799, 14800, 14801, 14802, 14803, 14804, 14805, 14806, 14807, 14808, 14809, 14810, 14811, 14812, 14813, 14814, 14815, 14816]


In [72]:
print(target_mate.get_reference_positions())

[15025, 15026, 15027, 15028, 15029, 15030, 15031, 15032, 15033, 15034, 15035, 15036, 15037, 15795, 15796, 15797, 15798, 15799, 15800, 15801, 15802, 15803, 15804, 15805, 15806, 15807, 15808, 15809, 15810, 15811, 15812, 15813, 15814, 15815, 15816, 15817, 15818, 15819, 15820, 15821, 15822, 15823, 15824, 15825, 15826, 15827, 15828, 15829, 15830, 15831, 15832, 15833, 15834, 15835, 15836, 15837, 15838, 15839, 15840, 15841, 15842, 15843, 15844, 15845, 15846, 15847, 15848, 15849, 15850, 15851, 15852, 15853, 15854, 15855, 15856, 15857, 15858, 15859, 15860, 15861, 15862, 15863, 15864, 15865, 15866, 15867, 15868, 15869, 15870, 15871, 15872, 15873, 15874, 15875, 15876, 15877, 15878, 15879, 15880, 15881, 15882]


In [73]:
target_mate.get_reference_sequence()

'CTCGTGGCTCTCACTTGCTCCTGCTCCTTCTGCTGCTGCTTCTCCAGCTTTCGCTCCTTCATGCTGCGCAGCTTGGCCTTGCCGATGCCCCCAGCTTGGCG'

In [87]:
target_mate.get_tags()

[('NH', 1),
 ('HI', 1),
 ('NM', 0),
 ('MD', '101'),
 ('AS', 201),
 ('XS', '-'),
 ('RG',
  'RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01')]

In [88]:
target_read.infer_query_length()

101

In [89]:
target_read.infer_read_length()

101

In [90]:
target_read.is_duplicate

False

In [92]:
target_read.is_paired

True

In [93]:
target_read.is_proper_pair

True

In [94]:
target_read.is_qcfail

False

In [95]:
target_read.is_read1

True

In [97]:
target_mate.is_read2

True

In [98]:
target_mate.is_reverse

True

In [99]:
target_read.is_secondary

False

In [100]:
target_read.is_supplementary

False

In [101]:
target_read.is_unmapped

False

In [102]:
target_read.mapping_quality

255

In [103]:
target_read.mate_is_reverse

True

In [104]:
target_read.mate_is_unmapped

False

In [105]:
target_read.next_reference_id

0

In [106]:
target_read.next_reference_name

'1'

In [107]:
target_read.next_reference_start

15025

In [112]:
print(target_mate.query_alignment_end)
print(target_mate.query_alignment_length)

101
101


In [114]:
target_read.query_alignment_qualities

array('B', [31, 31, 30, 35, 37, 37, 37, 37, 39, 39, 33, 35, 39, 40, 38, 39, 40, 40, 40, 41, 40, 41, 36, 40, 38, 36, 39, 32, 35, 39, 38, 38, 40, 38, 33, 39, 35, 39, 38, 27, 36, 39, 37, 24, 26, 30, 34, 31, 31, 26, 26, 29, 29, 34, 35, 31, 34, 33, 33, 33, 26, 26, 18, 26, 33, 16, 24, 27, 27, 31, 32, 33, 24, 34, 34, 24, 27, 31, 33, 29, 31, 35, 16, 15, 10, 23, 32, 31, 32, 30, 23, 30, 33, 22, 7, 23, 19, 29, 34, 18, 29])

In [115]:
target_read.query_alignment_sequence

'GTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT'

In [116]:
target_read.query_alignment_start

0

In [117]:
target_read.query_length

101

In [118]:
target_read.query_name

'HWI-ST672:204:D1HPJACXX:6:1106:19494:193291'

In [119]:
target_read.query_qualities

array('B', [31, 31, 30, 35, 37, 37, 37, 37, 39, 39, 33, 35, 39, 40, 38, 39, 40, 40, 40, 41, 40, 41, 36, 40, 38, 36, 39, 32, 35, 39, 38, 38, 40, 38, 33, 39, 35, 39, 38, 27, 36, 39, 37, 24, 26, 30, 34, 31, 31, 26, 26, 29, 29, 34, 35, 31, 34, 33, 33, 33, 26, 26, 18, 26, 33, 16, 24, 27, 27, 31, 32, 33, 24, 34, 34, 24, 27, 31, 33, 29, 31, 35, 16, 15, 10, 23, 32, 31, 32, 30, 23, 30, 33, 22, 7, 23, 19, 29, 34, 18, 29])

In [120]:
target_read.query_sequence

'GTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT'

In [121]:
target_read.reference_end

14817

In [122]:
target_read.reference_length

101

In [124]:
target_mate.reference_length

858

In [125]:
target_read.reference_start

14716

In [126]:
target_read.template_length

1167

In [128]:
target_read.to_dict()

{'name': 'HWI-ST672:204:D1HPJACXX:6:1106:19494:193291',
 'flag': '99',
 'ref_name': '1',
 'ref_pos': '14717',
 'map_quality': '255',
 'cigar': '101M',
 'next_ref_name': '=',
 'next_ref_pos': '15026',
 'length': '1167',
 'seq': 'GTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT',
 'qual': '@@?DFFFFHHBDHIGHIIIJIJEIGEHADHGGIGBHDHG<EHF9;?C@@;;>>CD@CBBB;;3;B19<<@AB9CC9<@B>@D10+8A@A?8?B7(84>C3>',
 'tags': ['NH:i:1',
  'HI:i:1',
  'NM:i:0',
  'MD:Z:101',
  'AS:i:201',
  'XS:A:-',
  'RG:Z:RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01']}

In [129]:
target_read.to_string()

'HWI-ST672:204:D1HPJACXX:6:1106:19494:193291\t99\t1\t14717\t255\t101M\t=\t15026\t1167\tGTCCTCCTCTGCCTGTGGCTGCTGCGGTGGCGGCAGAGGAGGGATGGAGTCTGACACGCGGGCAAAGGCTCCTCCGGGCCCCTCACCAGCCCCAGGTCCTT\t@@?DFFFFHHBDHIGHIIIJIJEIGEHADHGGIGBHDHG<EHF9;?C@@;;>>CD@CBBB;;3;B19<<@AB9CC9<@B>@D10+8A@A?8?B7(84>C3>\tNH:i:1\tHI:i:1\tNM:i:0\tMD:Z:101\tAS:i:201\tXS:A:-\tRG:Z:RIKEN:63a26562-e1fb-11e4-9899-a52c75ff8f68:icgc_rna_RNA_RK054_liver_20120731_01'

### FASTA:

In [5]:
genome = pysam.FastaFile('/home/leo/BioData/genomes/GRCh37.p11.genome.fa')

In [43]:
genome.references

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrX',
 'chrY',
 'chrM',
 'GL877870.2',
 'GL877872.1',
 'GL383535.1',
 'JH159133.1',
 'KB021647.1',
 'JH159131.1',
 'GL949745.1',
 'JH720447.1',
 'GL582968.1',
 'JH720446.1',
 'JH591181.2',
 'JH720443.1',
 'JH159134.2',
 'JH636052.4',
 'JH636053.2',
 'JH636054.1',
 'JH636056.1',
 'JH636058.1',
 'JH636057.1',
 'JH720451.1',
 'JH720452.1',
 'JH720453.1',
 'JH720454.2',
 'JH159136.1',
 'JH806587.1',
 'JH806588.1',
 'JH806589.1',
 'JH806590.2',
 'JH806591.1',
 'JH806592.1',
 'JH806593.1',
 'JH806594.1',
 'JH806595.1',
 'JH806596.1',
 'JH806597.1',
 'JH720448.1',
 'JH806598.1',
 'JH806599.1',
 'JH806600.1',
 'JH806601.1',
 'JH806602.1',
 'JH806573.1',
 'JH806574.1',
 'JH806575.1',
 'JH806580.1',
 'JH806583.1',
 'JH806584.1',
 'JH806585.1',
 'JH806603.1',
 'JH159150.2',
 'GL582969.1

In [7]:
import pandas as pd

In [44]:
df = pd.read_table('sorted_output/perl_PCAWG.sorted.tsv', header=None)

In [51]:
junctions = df.iloc[:, 0]

In [52]:
junctions = list(junctions.apply(lambda x: x.split('_')))

In [53]:
unique = set()
for junction in junctions:
    unique.add(junction[0])

In [54]:
junctions = list(filter(lambda x: x[0][0].isdigit() or x[0][0] in ('M', 'X', 'Y'), junctions))

In [55]:
junctions = list(map(lambda x: ('chr' + x[0] if x[0][0] != 'M' else 'chrM', int(x[1]), int(x[2])), junctions))

In [56]:
%%timeit
sites = []
for junction in junctions:
    x = genome.fetch(junction[0], junction[1], junction[1] + 2) + \
        genome.fetch(junction[0], junction[1] - 2, junction[1])
    sites.append(x)

8.56 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
sites = []
for junction in junctions:
    x = genome.fetch(junction[0], junction[1], junction[1] + 2) + \
        genome.fetch(junction[0], junction[1] - 2, junction[1])
    sites.append(x)